#### Tuesday, December 19, 2023

Actually running this today ... 

Attempted to step through this notebook but was having problems with making the call to Google Generative Language API ... 

#### Thursday, December 7, 2023

[Advanced RAG 06 - RAG Fusion](https://www.youtube.com/watch?v=GchC5WxeXGc)

https://colab.research.google.com/drive/1rTO2qtBFw4voPr3yyvs9uNsIaGT7da0y?usp=sharing

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# RAG Fusion

In [ ]:
!pip -q install langchain huggingface_hub openai tiktoken pypdf
!pip -q install google-generativeai chromadb unstructured
!pip -q install sentence_transformers
!pip -q install -U FlagEmbedding


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Download the Data & Utils

In [2]:
import os
import requests
import zipfile
from io import BytesIO
import textwrap

def download_and_extract_zip(url, target_folder):
    # Ensure the target folder exists
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Download the file from the URL
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {url}")

    # Unzip the file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(target_folder)

    print(f"Files extracted to {target_folder}")


def zip_folder(folder_path, zip_file_path):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a path relative to the folder to avoid storing absolute paths
                relative_path = os.path.relpath(os.path.join(root, file), os.path.dirname(folder_path))
                # Add file to the zip file
                zipf.write(os.path.join(root, file), arcname=relative_path)

    print(f"{zip_file_path} created successfully.")



def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

# URL of the zip file
url = "https://www.dropbox.com/scl/fi/av3nw07o5mo29cjokyp41/singapore_text_files_languages.zip?rlkey=xqdy5f1modtbnrzzga9024jyw&dl=1" # Ensure dl=1 for direct download

# Folder to save extracted files
folder = "singapore_text"

# Call the function
download_and_extract_zip(url, folder)


Files extracted to singapore_text


In [3]:
## download the chroma DB ... this creates a 'chroma_db' folder
url = 'https://www.dropbox.com/scl/fi/3kep8mo77h642kvpum2p7/singapore_chroma_db.zip?rlkey=4ry4rtmeqdcixjzxobtmaajzo&dl=1'
download_and_extract_zip(url, '.')

Files extracted to .


In [4]:
import os
from getpass import getpass

# enter your api key
GOOGLE_API_KEY = getpass("Enter your API key: ")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
!pip show langchain

Name: langchain
Version: 0.0.350
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: None
Author-email: None
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: langchain-core, pydantic, async-timeout, aiohttp, PyYAML, jsonpatch, langsmith, numpy, requests, dataclasses-json, langchain-community, SQLAlchemy, tenacity
Required-by: 


## Google

## Imports

In [6]:
from langchain.llms import GooglePalm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
import langchain



## Load in Docs

In [7]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [8]:
%%time
loader = DirectoryLoader('singapore_text/Textfiles3/English/', glob="*.txt", show_progress=True)
docs = loader.load()

100%|██████████| 646/646 [00:22<00:00, 29.36it/s]

CPU times: user 21.1 s, sys: 929 ms, total: 22 s
Wall time: 22.1 s


In [9]:
len(docs)
# docs = docs[:10]
len(docs)

646

In [10]:
docs[0]

Document(page_content="Link: https://www.visitsingapore.com/editorials/explore\n\nsingapores\n\nislands/\n\nTitle: Explore Singapore’s islands\n\nSo you’ve shopped till you dropped at Orchard Road, taken your mandatory selfie with the Merlion, and marvelled at the skyscrapers at Marina Bay. Want to see more of Singapore? Head to these offshore islands for a Singaporean experience unlike any other.\n\nSt John’s Island\n\nCalling all cat lovers, skip the cat cafes and head to St John's Island for some quality time with your favourite furry friends—the island has become famous in recent years for the many friendly felines lounging around.\n\nSt John’s Island has also seen its fair share of history. What was once a quarantine centre for infectious diseases and later a drug rehabilitation centre, was transformed in 1975. The island was given a new lease of life, as an oasis where travellers enjoy swimming lagoons, beaches, picnic grounds, trekking routes and more.\n\nHow to get there: Catch

In [11]:
raw_text = ''
for i, doc in enumerate(docs):
    text = doc.page_content
    if text:
        raw_text += text

In [12]:
len(raw_text)

2123598

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)

In [14]:
texts = text_splitter.split_text(raw_text)

In [15]:
len(texts)

6210

In [16]:
texts[1]

"St John’s Island\n\nCalling all cat lovers, skip the cat cafes and head to St John's Island for some quality time with your favourite furry friends—the island has become famous in recent years for the many friendly felines lounging around."

## BGE Embeddings

In [17]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs,

)

## Vector DB

In [20]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 Intel_neural_chat_7b_v3_1_+_4_8bit_Basics.ipynb
 YT_Chatting_with_Flan20B_UL2_using_LangChain_Chatbot.ipynb
 YT_LangChain_Advanced_RAG_tips_and_Tricks_04_Contextual_Compression_+_Filtering.ipynb
 YT_LangChain_RAG_tips_and_Tricks_01_Self_Query.ipynb
 YT_LangChain_RAG_tips_and_Tricks_02_Parent_Document_Retriever.ipynb
'YT_LangChain_RAG_tips_and_Tricks_03_BM25_+_Ensemble_=_Hybrid_Search.ipynb'
 YT_LangChain_RAG_tips_and_Tricks_05_HyDE.ipynb
 YT_LangChain_Running_HuggingFace_Models_Locally.ipynb
 YT_Langchain_Tools_Chains_PAL_.ipynb
'YT_RagFusion_MakerSuite_PaLM_2_Langchain_with_Chroma_&_BGE.ipynb'
 blogposts
 chroma_db
 singapore_text


In [21]:
# %%time
### Make the chroma and persiste to disk
db = Chroma.from_texts(texts,
                       embedding_function,
                       persist_directory="persistDb")


In [18]:
# %%time
### Make the chroma and persiste to disk
# db = Chroma.from_texts(texts,
#                        embedding_function,
#                        persist_directory="./chroma_db")



### load from disk
# db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
#db = Chroma(persist_directory="chroma_db", embedding_function=embedding_function)





In [ ]:
### Save to zip
# zip_folder('/content/chroma_db', 'chroma_db.zip')

chroma_db.zip created successfully.


In [22]:
query = "Tell me about Universal Studios Singapore?"

db.similarity_search(query, k=5)

[Document(page_content='Universal Studios Singapore'),
 Document(page_content='Universal Studios Singapore is a popular theme park that offers thrilling rides and entertainment for all ages. You can buy Universal Studios Singapore tickets and enjoy an unbeatable discount of 40% from MySingaporePass. The pass provides a hassle-free booking process, allowing you to skip the long queues and gain easy access to the park. Enjoy exhilarating roller coasters, live shows, and immersive themed zones inspired by your favorite movies. The discounted entry with My Singapore Pass'),
 Document(page_content='Cons: If you are not into nightlife this area might not be the right one for you to stay.Link: https://www.visitsingapore.com/see\n\ndo\n\nsingapore/recreation\n\nleisure/fun\n\nthings\n\nto\n\ndo/universal\n\nstudios\n\nsingapore/\n\nTitle: Universal Studios Singapore\n\nThe shimmering wonders of the silver screen comes to vivid life at Universal Studios Singapore, the first-ever Hollywood movie

## Setup a Retriever

In [23]:
retriever = db.as_retriever(k=5) # can add mmr fetch_k=20, search_type="mmr"

retriever.get_relevant_documents(query)[1]

Document(page_content='Universal Studios Singapore is a popular theme park that offers thrilling rides and entertainment for all ages. You can buy Universal Studios Singapore tickets and enjoy an unbeatable discount of 40% from MySingaporePass. The pass provides a hassle-free booking process, allowing you to skip the long queues and gain easy access to the park. Enjoy exhilarating roller coasters, live shows, and immersive themed zones inspired by your favorite movies. The discounted entry with My Singapore Pass')

## Chat chain

In [24]:
from operator import itemgetter

from langchain.chat_models import ChatGooglePalm

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

In [26]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatGooglePalm()

In [27]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

To get this next cell to run, I had to pip install google-generativeai and then go to the google developers console and Enable the Generative Language API ... otherwise it just kept firing back 'SERVICE DISABLED' ...

In [29]:
text_reply = chain.invoke("Tell me about Universal Studio Singapore")

print(wrap_text(text_reply))

Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised FailedPrecondition: 400 User location is not supported for the API use..
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised FailedPrecondition: 400 User location is not supported for the API use..
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised FailedPrecondition: 400 User location is not supported for the API use..


KeyboardInterrupt: 

## With RagFusion

In [30]:
from langchain.chat_models import ChatGooglePalm
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

In [31]:
prompt = ChatPromptTemplate(input_variables=['original_query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])

In [32]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),
#     ("user", "Generate multiple search queries related to: {question}/n OUTPUT (4 queries):"),
# ])

prompt

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])

In [33]:
generate_queries = (
    prompt | ChatGooglePalm(temperature=0) | StrOutputParser() | (lambda x: x.split("\n"))
)

In [34]:
original_query = "universal studios Singapore"

In [36]:
from langchain.load import dumps, loads


# def reciprocal_rank_fusion(results: list[list], k=60):
def reciprocal_rank_fusion(results, k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [37]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [38]:
langchain.debug = True

In [39]:
ragfusion_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [40]:
ragfusion_chain.invoke({"question": original_query})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "universal studios Singapore"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "universal studios Singapore"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are a helpful assistant that generates multiple search queries based on a single input query.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          

Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised FailedPrecondition: 400 User location is not supported for the API use..
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised FailedPrecondition: 400 User location is not supported for the API use..
Retrying langchain_community.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised FailedPrecondition: 400 User location is not supported for the API use..


[llm/error] [1:chain:RunnableSequence > 3:llm:ChatGooglePalm] [12.79s] LLM run errored with error:
"KeyboardInterrupt()"
[chain/error] [1:chain:RunnableSequence] [12.79s] Chain run errored with error:
"KeyboardInterrupt()"


KeyboardInterrupt: 

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
full_rag_fusion_chain.input_schema.schema()

{'title': 'RunnableParallelInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [ ]:
full_rag_fusion_chain.invoke({"question": "Tell me about Universal Studio Singapore?"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "me

"Universal Studios Singapore is a theme park located on Sentosa Island in Singapore. It is the first Universal Studios theme park in Southeast Asia. The park opened on 18 September 2010.\n\nUniversal Studios Singapore is divided into seven themed zones:\n\n* Hollywood\n* New York\n* Sci-Fi City\n* Ancient Egypt\n* The Lost World\n* Far Far Away\n* Madagascar\n\nThe park features a variety of rides, shows, and attractions based on popular movies and television shows. Some of the most popular rides include:\n\n* TRANSFORMERS The Ride: The Ultimate 3D Battle\n* Jurassic Park Rapids Adventure\n* Battlestar Galactica: HUMAN vs. CYLON™\n* Revenge of the Mummy: The Ride\n* Shrek 4-D Adventure\n* Sesame Street Spaghetti Space Chase\n\nThe park also features a number of live shows, including:\n\n* Universal's Animal Actors\n* The Universal's Superstar Parade\n* The Universal's Cinematic Spectacular: 100 Years of Movie Memories\n\nUniversal Studios Singapore is a popular tourist destination and 